In [ ]:
# 패키지 설치
!pip install yfinance --quiet

In [ ]:
# 패키지 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
sns.set()

In [ ]:
# 데이터 다운로드
tickers = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
etf = yf.Tickers(tickers)
data = etf.history(start='2011-01-01', actions=False)
data.drop(['Open', 'High', 'Low', 'Volume'], inplace=True, axis=1)
data = data.droplevel(0, axis=1).resample('M').last()

In [ ]:
# 수익률 데이터
ret = data.pct_change().dropna()

# 공분산 행렬
cov_mat = ret.cov() * 12

In [ ]:
# 공분산 행렬 고윳값 분해
w, v = np.linalg.eig(cov_mat)

In [ ]:
v

In [ ]:
w

In [ ]:
# 공분산 행렬 히트맵
plt.figure(figsize=(10, 8))
sns.heatmap(cov_mat, annot=True)

In [ ]:
# 공분산 행렬의 역행렬 계산 함수
def inverse_cov_mat(cov_mat, eps=0.05, is_plot=True):

    # 고윳값 분해
    w, v = np.linalg.eig(cov_mat)
    
    # 고윳값이 음수라면 0으로 치환
    if np.where(w >= 0, True, False).sum() != len(w):
        w = np.where(w < 0, 0, w)

    # 유의미하지 않는 고윳값을 0으로 처리
    weighted_w = w / np.sum(w)
    noise_free_w = np.where(weighted_w >= eps, w, 0)

    # 고윳값과 상대적 가중치에 대한 그래프 생성
    if is_plot:
        plt.plot(np.sort(w)[::-1], marker='o', label='eigenvalue')
        plt.plot(np.sort(noise_free_w)[::-1], marker='x', label='updated eigenvalue')
        plt.legend()
        plt.show()

    # 역행렬 계산
    inv_mat = v @ np.diag(np.where(noise_free_w != 0, 1 / noise_free_w, 0)) @ v.T
    new_cov_mat = v @ np.diag(noise_free_w) @ v.T

    return inv_mat, new_cov_mat

In [ ]:
# 업데이트된 공분산 행렬
inv_mat, new_cov_mat = inverse_cov_mat(cov_mat)

In [ ]:
# 공분산 행렬 히트맵
plt.figure(figsize=(10, 8))
sns.heatmap(new_cov_mat, annot=True)

Copyright 2022. 퀀트대디. All rights reserved.